In [ ]:
# 원본은 내 pc(Dell)의 deepcell-speechbrain-2 Docker에 있음, file name : inference.ipynb
# KsponSpeech data로 테스트

import os
import glob
from pathlib import Path
from tqdm import tqdm
import pandas as pd
from speechbrain.pretrained import EncoderDecoderASR

In [ ]:
# cpu로 추론, GPU가 있으면 GPU로 해야 빠름

asr_model = EncoderDecoderASR.from_hparams(
    source="ddwkim/asr-conformer-transformerlm-ksponspeech", 
    savedir="pretrained_models/asr-conformer-transformerlm-ksponspeech")

In [ ]:
audio_0 = "/data/ksponmini/test/eval_clean/KsponSpeech_E00099.wav"
trs = asr_model.transcribe_file(audio_0)
trs

# -> '반말을 할 수가 없어'

In [ ]:
audio_dir = '/data/ksponmini/test/eval_clean/*'

file_list = glob.glob(audio_dir) # audio file path
file_list_wav = [file for file in file_list if file.endswith(".wav")] # wav 파일만 모아서 리스트 만들기

In [ ]:
# 오디오 파일을 텍스트 파일로 변화

trans = []
for file in tqdm(file_list_wav):
    # print(os.path.basename(file))
    # print(file)
    file_name = Path(file).stem # 확장자 제거
    id = file_name.split('_')[1] # _ 앞부분 제거하여 ID만 추출
    trs = asr_model.transcribe_file(file)   # 음성인식
    trans.append([id, trs])
    # print(id)

trans[:2] # 앞의 2개로 확인

In [ ]:
# 리스트롤 csv 파일로 저장
hyp_file = 'hyp.csv'
with open(hyp_file, 'w') as f:
    for item in trans:
        # write each item on a new line
        f.write(f"{item[0]},{item[1]}\n")
f.close()

In [ ]:
hyp_df = pd.DataFrame(trans, columns = ['spk_id', 'hyp'])
# hyp_df = pd.read_csv(hyp_file, names=['spk_id', 'hyp'], header=None)

In [ ]:
test_file = '/speechbrain/recipes/KsponMini/Tokenizer/results/5K_subword_unigram_LM/eval_clean.csv'
test_df = pd.read_csv(test_file)

In [ ]:
# hyp_df와 test_df를 합치기
merged_df = pd.merge(hyp_df, test_df, how='inner', on='spk_id')

In [ ]:
hyp_label_df = merged_df[['spk_id', 'hyp', 'wrd']]

In [ ]:
hyp_label_df.to_csv('hyp_label.csv', index=False)